# 使用pytorch实现



In [ ]:
from IPython import display
%matplotlib inline
import random
import torch
from d2l import torch as d2l
from torch import nn
from matplotlib import pyplot as plt
from torch.utils import data


## 从零开始实现

### 构建数据集
> $y = Xw + b + \gamma$

In [ ]:
def syntheic_data(w, b, num_examples):
    """生成y = Xw + b + 噪音"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

### 真实参数

In [ ]:
true_w = torch.tensor([-2, -3.4])
true_b = 4.2
features, labels = syntheic_data(true_w, true_b, 1000)

print('features:', features[0], '\nlabel:', labels[0])

### 可视化散点图

In [ ]:
def use_svg_display():
    """使用svg格式"""
    display.set_matplotlib_formats('svg')


def set_figsize(figsize=(3.5, 2.5)):
    """设置图片大小"""
    use_svg_display()
    plt.rcParams['figure.figsize'] = figsize

# 画图
plt.scatter(features[:, 1].detach().numpy(),  # 旧版本pytorch转numpy需要detach先
                labels.detach().numpy(), 1)


### 随机读取样本

In [ ]:
def data_iter(batch_size, features, labels):
   num_examples = len(features)  # 样本数
   # 这些样本是随机读取的
   indices = list(range(features))
   random.shuffle(indices)
   for i in range(0, num_examples, batch_size):
       batch_indices = torch.tensor(
           indices[i: min(i + batch_size, num_examples)])
       yield features[batch_indices], labels[batch_indices]

batch_size = 10

for X, y in data_iter(batch_size,  features, labels):
    print(X, '\n', y)
    break


### 初始化模型参数

In [ ]:
# 初始化模型参数
w = torch.normal(0, 0.01, size=(2, 1),requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 定义模型
def linreg(X, w, b):
    """线性回归模型"""
    return torch.matmul(X, w)+  b


# 定义损失函数
def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# 定义优化算法
def sgd(params, lr, batch_size):
    """小批量梯度下降"""
    with torch.no_grad():  # 更新的时候不要参与梯度计算
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()  # 下一次梯度的时候就不跟上一次相关了


### 训练过程

In [ ]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y) # `X`和`y`的小批量损失
        # 计算梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    # 评价一下
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')


print(f'w的估计误差:{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差:{true_b - b}')


## 调用pytorch自带实现

In [ ]:
features, labels = d2l.synthetic_data(true_w, true_b, num_examples=1000)

def load_array(data_arrays, batch_size, is_train= True):
    """构造一个pytorch的数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)


# 转成python的iter
next(iter(data_iter))

In [ ]:
# 单层神经网络
net = nn.Sequential(nn.Linear(2, 1))

In [ ]:
# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].weight.data.fill_(0)

# 损失函数
loss = nn.MSELoss

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [ ]:
# 开始训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # 自带模型参数，不需要w和b放进去了
        trainer.zero_grad()  # 优化器梯度清零
        l.backward()  # 自动帮你求sum了
        trainer.step()  # 模型更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')